In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/leonidesguerra/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [3]:
news_url='https://redplanetscience.com/'
img_url='https://spaceimages-mars.com/'
facts_url='https://galaxyfacts-mars.com/'
hemisphere_url='https://marshemispheres.com/'

In [4]:
#NASA Mars News
browser.visit(news_url)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")


In [5]:
#Obtaining latest news and paragraph
news=soup.find('div', id = 'news')
latest_news_title=news.find('div', class_='content_title').text
latest_news_par=news.find('div', class_='article_teaser_body').text

In [6]:
# JPL Mars Space Images - Featured Image
browser.visit(img_url)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")

In [7]:
browser.links.find_by_partial_text('FULL IMAGE').click()

In [8]:
images=soup.find_all('img')
mars_image=images[1]['src']
featured_image_url=img_url+mars_image
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [9]:
#scrape tables from mars facts url
tables = pd.read_html(facts_url)
facts=tables[1]

In [10]:
facts

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [11]:
#converting the fact table to html and removing both header and index
facts_html=facts.to_html(header =False, index =False)
fact_table=facts_html.replace('\n', '')
fact_table

'<table border="1" class="dataframe">  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [12]:
#gather hemisphere image url and title for each hemisphere
browser.visit(hemisphere_url)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")

In [13]:
# findig the div that contains the title for each image and its url, and
# creating a list of urls in order to acces each high resolution image page
items=soup.find_all('div', class_='item')
link=[]
for item in items:
    l=item.find('div', class_='description')
    li=hemisphere_url+l.a['href']
    link.append(li)

hemisphere_images=[]

In [14]:
link

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [15]:
# obtaining the high resolution image url for each img and generation 
# a list of dictionaries that contain both the title and img url for each image
for idx, item in enumerate(items):
    title=item.find('h3').text
    browser.visit(link[idx])
    result= browser.find_by_tag('a')
    img_url= result[3]['href']
    hemisphere_images.append({'title': title, 'img_url': img_url})
   

In [16]:
hemisphere_images

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [17]:
# quitting the browser after scraping
browser.quit()